In [1]:
pip install pandas pyarrow fsspec s3fs


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "70e208d9d8ba1534136297fb1f3fe396" # Replace with your actual API key


provinces ={
    "Satitram Alumni": {
        "lat": 13.752916,
        "lon": 100.618616
    },
    "Siam Paragon": {
        "lat": 13.746239,
        "lon": 100.534345
    },
    "MBK Center": {
        "lat": 13.746141,
        "lon": 100.530547
    },
    "Chulalongkorn University": {
        "lat": 13.746724,
        "lon": 100.530898
    },
    "Erawan Shrine": {
        "lat": 13.746656,
        "lon": 100.541333
    },
    "CentralWorld": {
        "lat": 13.746774,
        "lon": 100.539462
    },
    "Pratunam Market": {
        "lat": 13.749609,
        "lon": 100.539115
    },
    "Jim Thompson House": {
        "lat": 13.746603,
        "lon": 100.529531
    },
    "Bangkok Art and Culture Centre": {
        "lat": 13.746599,
        "lon": 100.531101
    },
    "Lumphini Park": {
        "lat": 13.727924,
        "lon": 100.542287
    },
    "Wat Arun": {
        "lat": 13.743682,
        "lon": 100.488146
    },
    "Wat Pho": {
        "lat": 13.746697,
        "lon": 100.493469
    },
    "Grand Palace": {
        "lat": 13.750046,
        "lon": 100.491346
    },
    "Asiatique The Riverfront": {
        "lat": 13.703379,
        "lon": 100.509145
    },
    "ICONSIAM": {
        "lat": 13.723621,
        "lon": 100.517333
    },
    "Khao San Road": {
        "lat": 13.749136,
        "lon": 100.495136
    },
    "Terminal 21": {
        "lat": 13.736657,
        "lon": 100.561172
    },
    "The Mall Bangkapi": {
        "lat": 13.767090,
        "lon": 100.640134
    },
    "Dusit Zoo": {
        "lat": 13.766410,
        "lon": 100.525019
    },
    "Sukhumvit Road": {
        "lat": 13.731731,
        "lon": 100.571259
    }
}
# Function to fetch and process weather data
def get_weather_data(province="Satitram Alumni"):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric",
        "lang": "th"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)


        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'province': province,
            'location_name': location,
            'api_location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp'],
            'main.humidity': data['main']['humidity'],
            'wind.speed': data['wind']['speed'],
            'precipitation': get_precipitation(data)
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [3]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df

NameError: name 'location' is not defined

In [148]:
dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)
print(dt) 

2025-05-12 07:18:27.799970+06:42


In [149]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [150]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour'],
    
)

# test read parquet files

In [4]:
pip install fsspec


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [171]:
import s3fs
import pandas as pd

fs = s3fs.S3FileSystem(
    key=ACCESS_KEY,
    secret=SECRET_KEY,
    client_kwargs={'endpoint_url': 'http://lakefs-dev:8000'}
)

# ค้นหาเฉพาะไฟล์ .parquet ที่มีอยู่จริง
all_files = fs.find('weather/main/weather.parquet/')
parquet_files = [f for f in all_files if f.endswith('.parquet')]

print("จำนวนไฟล์ที่พบ:", len(parquet_files))

# โหลดทีละไฟล์ แล้วรวมกัน
dfs = []
for f in parquet_files:
    try:
        df = pd.read_parquet(f's3a://{f}', storage_options=storage_options)
        dfs.append(df)
    except FileNotFoundError:
        print(f"ข้ามไฟล์: {f} (ไม่พบ)")
    except Exception as e:
        print(f"ข้ามไฟล์: {f} ({e})")

# รวม DataFrame ทั้งหมด
if dfs:
    df_all = pd.concat(dfs, ignore_index=True)
    df_all.info()
    df_all.head()
else:
    print("ไม่พบไฟล์ .parquet ที่สามารถโหลดได้")

จำนวนไฟล์ที่พบ: 645
ข้ามไฟล์: weather/main/weather.parquet/year=2025/month=5/day=8/hour=6/dd1741ec35434825a62561e9625ed3e8-0.parquet (ไม่พบ)
ข้ามไฟล์: weather/main/weather.parquet/year=2025/month=5/day=8/hour=6/fa603956177e4588a528db9e2bb9dded-0.parquet (ไม่พบ)
ข้ามไฟล์: weather/main/weather.parquet/year=2025/month=5/day=9/hour=9/0cc55943c49d469b97dd8d74f182c783-0.parquet (ไม่พบ)
ข้ามไฟล์: weather/main/weather.parquet/year=2025/month=5/day=9/hour=9/c5ae3955bdd34227ade50b4a760b04e3-0.parquet (ไม่พบ)
ข้ามไฟล์: weather/main/weather.parquet/year=2025/month=5/day=9/hour=9/dd85d7c03e914d6c87056e21cc7b6cac-0.parquet (ไม่พบ)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   timestamp            2557 non-null   datetime64[ns]              
 1   minute               2555 non-null   float64      

In [174]:
import pyarrow.dataset as ds

dataset = ds.dataset(
    "weather/main/weather.parquet",  # <== ตัด "s3a://" ออก
    format="parquet",
    partitioning="hive",
    filesystem=fs  # <== fs ที่สร้างด้วย s3fs
)

table = dataset.to_table()
df_all = table.to_pandas()

df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574 entries, 0 to 2573
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   timestamp            2574 non-null   datetime64[ns]              
 1   minute               2572 non-null   float64                     
 2   created_at           2572 non-null   datetime64[ns, Asia/Bangkok]
 3   requested_province   2508 non-null   object                      
 4   location             2574 non-null   object                      
 5   weather_main         2572 non-null   object                      
 6   weather_description  2572 non-null   object                      
 7   main.temp            2508 non-null   float64                     
 8   year                 2574 non-null   int32                       
 9   month                2574 non-null   int32                       
 10  day                  2574 non-null  

,timestamp,minute,created_at,requested_province,location,weather_main,weather_description,main.temp,year,month,day,hour
0,2025-05-01 11:31:10.350466,31.0,2025-05-01 11:49:10.333970+07:00,Pathum Thani,Pathum Thani,Clouds,few clouds,33.15,2025,5,1,11
1,2025-05-01 11:31:10.717529,31.0,2025-05-01 11:49:10.717493+07:00,Bangkok,Bangkok,Clouds,overcast clouds,33.94,2025,5,1,11
2,2025-05-01 11:31:11.057869,31.0,2025-05-01 11:49:11.057842+07:00,Chiang Mai,Chiang Mai,Clouds,few clouds,31.28,2025,5,1,11
3,2025-05-01 11:31:11.341738,31.0,2025-05-01 11:49:11.341696+07:00,Phuket,Kathu,Rain,moderate rain,25.92,2025,5,1,11
4,2025-05-01 11:45:52.508233,45.0,2025-05-01 12:03:52.508159+07:00,Pathum Thani,Pathum Thani,Clouds,few clouds,32.89,2025,5,1,11


In [180]:
path_single_partition = 's3a://weather/main/weather.parquet/year=2025/month=5/day=13/hour=3/3f5d0f9a031c4b0193cbcba94a518b4b-0.parquet'

df2=pd.read_parquet(    
    path=path_single_partition,
    storage_options=storage_options,
)
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype                       
---  ------               --------------  -----                       
 0   timestamp            20 non-null     datetime64[ns]              
 1   minute               20 non-null     int64                       
 2   created_at           20 non-null     datetime64[ns, Asia/Bangkok]
 3   location             20 non-null     object                      
 4   temperature          20 non-null     float64                     
 5   feels_like           20 non-null     float64                     
 6   humidity             20 non-null     int64                       
 7   pressure             20 non-null     int64                       
 8   wind_speed           20 non-null     float64                     
 9   visibility           20 non-null     int64                       
 10  weather_main         20 non-null     obj

,timestamp,minute,created_at,location,temperature,feels_like,humidity,pressure,wind_speed,visibility,weather_main,weather_description,year,month,day,hour
0,2025-05-13 03:00:00.189672,0,2025-05-13 03:18:00.172396+07:00,Satitram Alumni,28.89,35.89,99,1009,3.38,10000,Clouds,เมฆเต็มท้องฟ้า,2025,5,13,3
1,2025-05-13 03:00:00.458039,0,2025-05-13 03:18:00.458028+07:00,Siam Paragon,28.84,35.84,99,1009,3.39,8181,Clouds,เมฆเต็มท้องฟ้า,2025,5,13,3
2,2025-05-13 03:00:00.764328,0,2025-05-13 03:18:00.764311+07:00,MBK Center,28.84,35.84,99,1009,3.39,8181,Clouds,เมฆเต็มท้องฟ้า,2025,5,13,3
3,2025-05-13 03:00:01.033206,0,2025-05-13 03:18:01.033194+07:00,Chulalongkorn University,28.84,35.84,99,1009,3.39,8181,Clouds,เมฆเต็มท้องฟ้า,2025,5,13,3
4,2025-05-13 03:00:01.322885,0,2025-05-13 03:18:01.322875+07:00,Erawan Shrine,28.83,35.83,99,1009,3.39,8181,Clouds,เมฆเต็มท้องฟ้า,2025,5,13,3


In [52]:
print(dt)

2025-05-09 08:43:12.246090+06:42


# Test Duck and Dask

In [175]:
pip install duckdb


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [177]:
import duckdb
import pandas as pd

# Connect to an in-memory DuckDB instance.
con = duckdb.connect(database=':memory:')

storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

# Use DuckDB's read_parquet() function to read the dataset.
# Assume the dataset is stored in the directory "output_parquet_dataset" with hive partitions:
# output_parquet_dataset/year=2025/month=02/day=17/...
query = """
INSTALL httpfs;
LOAD httpfs;

SET s3_endpoint='lakefs-dev:8001';
SET s3_access_key_id='access_key'; 
SET s3_secret_access_key='secret_key'; 
SET s3_url_style='path';
SET s3_use_ssl=false;

SELECT *
FROM read_parquet('s3a://weather/main/weather.parquet')
"""

df_duck = con.execute(query).df()

print("DuckDB Parquet Query Result:")
df_duck.info()
df_duck.head(20)

IOException: IO Error: Could not establish connection error for HTTP HEAD to 'http://lakefs-dev:8001/weather/main/weather.parquet' with status -50556336